In [ ]:
import cv2
import mediapipe as mp
import numpy as np

class handDetector():
    def __init__(self, 
                 mode=False, 
                 maxHands=1, 
                 modelComp=1, # 手掌偵測狀態初始化
                 detectionCon=0.5, 
                 trackCon=0.5):
        self.mode = mode
        self.maxHands = maxHands
        self.detectionCon = detectionCon
        self.trackCon = trackCon
        self.modelComp = modelComp
        
        self.mpHands = mp.solutions.hands # mediapipe 偵測手掌方法
        self.hands = self.mpHands.Hands(self.mode, 
                                        self.maxHands,
                                        self.modelComp,
                                        self.detectionCon, 
                                        self.trackCon) # mediapipe 啟用偵測手掌
        self.mpDraw = mp.solutions.drawing_utils # mediapipe 標記節點

    def findHands(self, img):
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) #偵測手掌方法是用RGB非BGR
        self.results = self.hands.process(imgRGB) # 偵測手掌
    
        if self.results.multi_hand_landmarks: #若有偵測到手掌(results.multi_hand_landmarks為True)
            for handLandmarks in self.results.multi_hand_landmarks: #對每隻手掌做特定動作
                self.mpDraw.draw_landmarks(img, handLandmarks, self.mpHands.HAND_CONNECTIONS) #對手掌做標記節點&連線
        
        return img
    
    def findPosition(self, img, handNumber):
        landmarkList = []
        if self.results.multi_hand_landmarks:
            myHand = self.results.multi_hand_landmarks[handNumber]
            
            for id, landmark in enumerate(myHand.landmark):
                height, width, channels = img.shape #channels為通道數(如RGB有3個channel)
                coordinate_x, coordinate_y = int(landmark.x * width), int(landmark.y * height)
                landmarkList.append([id, coordinate_x, coordinate_y]) #將節點的序號&座標放入回傳的序列中
                if id == 8:
                    cv2.circle(img, (coordinate_x, coordinate_y), 10, (0, 0, 255), cv2.FILLED) #節點處畫實心圓(食指節點顏色改變+變大)
                else:
                    cv2.circle(img, (coordinate_x, coordinate_y), 5, (0, 255, 0), cv2.FILLED) #節點處畫實心圓
            
        return landmarkList
        
    
def main():
    cap = cv2.VideoCapture(0)
    detector = handDetector()
    width, height = 1280, 720 #攝像頭寬&高大小設定
    cap.set(3, width) #id為3表示影片的寬
    cap.set(4, height) #id為4表示影片的高
    while True:
        retval, img = cap.read() #讀取攝像頭照到的影像，retval表示影像捕獲成功or失敗
        img = detector.findHands(img)
        landmarkList = detector.findPosition(img, 0)
        if len(landmarkList) != 0:
            print(landmarkList[8])
        
        img = np.flip(img, axis=1) #畫面左右翻轉
        cv2.namedWindow("Test", cv2.WINDOW_NORMAL)
        cv2.imshow("Test", img) #測試畫面
        key = cv2.waitKey(1) #等待1毫秒接收按鍵
        if key == 27: #若按下esc，退出迴圈
            break
    cap.release() #結束攝像機
    cv2.destroyAllWindows() #關閉所有視窗
    
if __name__ == "__main__": #預防其他ipynb檔案引用該檔案的函式時，不會執行該檔案的所有程式碼
    main() #執行該檔案時只會執行該函式